In [1]:
import numpy as np
class HiddenMarkovModel:
    """_summary_
    """

    def __init__(self, observation_states: np.ndarray, hidden_states: np.ndarray, prior_probabilities: np.ndarray, transition_probabilities: np.ndarray, emission_probabilities: np.ndarray):
        """_summary_

        Args:
            observation_states (np.ndarray): _description_
            hidden_states (np.ndarray): _description_
            prior_probabilities (np.ndarray): _description_
            transition_probabilities (np.ndarray): _description_
            emission_probabilities (np.ndarray): _description_
        """             
        self.observation_states = observation_states
        self.observation_states_dict = {observation_state: observation_state_index \
                                  for observation_state_index, observation_state in enumerate(list(self.observation_states))}

        self.hidden_states = hidden_states
        self.hidden_states_dict = {hidden_state_index: hidden_state \
                                   for hidden_state_index, hidden_state in enumerate(list(self.hidden_states))}
        

        self.prior_probabilities= prior_probabilities
        self.transition_probabilities = transition_probabilities
        self.emission_probabilities = emission_probabilities

In [304]:
import copy
import numpy as np
class ViterbiAlgorithm:
    """_summary_
    """    

    def __init__(self, hmm_object):
        """_summary_

        Args:
            hmm_object (_type_): _description_
        """              
        self.hmm_object = hmm_object

    def best_hidden_state_sequence(self, decode_observation_states: np.ndarray) -> np.ndarray:
        """_summary_

        Args:
            decode_observation_states (np.ndarray): _description_

        Returns:
            np.ndarray: _description_
        """        
        
        obs_st = self.hmm_object.observation_states
        obs_st_dict = self.hmm_object.observation_states_dict
        hid_st = self.hmm_object.hidden_states
        hid_st_dict = self.hmm_object.hidden_states_dict
        prior_probs = self.hmm_object.prior_probabilities
        emit_probs = self.hmm_object.emission_probabilities
        trans_probs = self.hmm_object.transition_probabilities

        # Initialize path (i.e., np.arrays) to store the hidden sequence states returning the maximum probability
        path = np.zeros((len(decode_observation_states), 
                         len(self.hmm_object.hidden_states)))
        path[0,:] = [hidden_state_index for hidden_state_index in range(len(self.hmm_object.hidden_states))]

        best_path = np.zeros(len(decode_observation_states)) 
        
        
        # Compute initial delta:
        # 1. Calculate the product of the prior and emission probabilities. This will be used to decode the first observation state.
        # 2. Scale
        init_ind = obs_st_dict[decode_observation_states[0]]
        delta = np.multiply(prior_probs, emit_probs.T[init_ind,:])
        #delta = delta / np.sum(delta)
        #best_path[0] = path[0, np.argmax(delta)]
        
        print("obs: ", decode_observation_states)

        # For each observation state to decode, select the hidden state sequence with the highest probability (i.e., Viterbi trellis)
        for trellis_node in range(1, len(decode_observation_states)):

            # TODO: comment the initialization, recursion, and termination steps
    
            current_obs = obs_st_dict[decode_observation_states[trellis_node]]
            #print("delt: ", delta)
            #print("trans: ", trans_probs.T)
            #print("emit: ", emit_probs[:,current_obs])

            product_of_delta_and_transition_emission = np.multiply(delta, trans_probs.T)
            product_of_delta_and_transition_emission = np.multiply(product_of_delta_and_transition_emission.T, emit_probs[:,current_obs])
            #print("prod: ", product_of_delta_and_transition_emission)

            # Select the hidden state sequence with the maximum probability
            max_hidden_ind = np.argmax(product_of_delta_and_transition_emission, axis=0) 
            max_hidden = np.max(product_of_delta_and_transition_emission, axis=0)
            #print("prod: ", product_of_delta_and_transition_emission)
            
            path[trellis_node,:] = max_hidden_ind
            #print("new path: ", path[trellis_node,:])
            
            #print("max hidden: ", max_hidden)
            best_path[trellis_node - 1] = np.argmax(max_hidden)
            print("best path: ", best_path)

            # Update best path
            #for hidden_state in range(len(self.hmm_object.hidden_states)):
            #    hid_st_dict[hidden_state]
            #    np.append(best_hidden_state_path, hidden_s)
                #path.append(max_hidden_ind)
            #    best_path[trellis_node] = max_hidden_ind
            #    print("trellis: ", path[trellis_node - 1, max_hidden_ind])
            #    best_path[trellis_node - 1] = path[trellis_node - 1, max_hidden_ind]
                
            
            # Update delta and scale
            delta = max_hidden
            
            #delta = np.multiply(max_hidden, emit_probs[current_obs,:])
            #delta = delta / np.sum(delta)
            
            # Set best hidden state sequence in the best_path np.ndarray THEN copy the best_path to path

        # Select the last hidden state, given the best path (i.e., maximum probability)
        #best_hidden_state_path = np.array([])
        print("final path: ", path)
        for state in range(len(decode_observation_states) - 1, 0, -1):
            best_path[state - 1] = path[state, int(best_path[state])]
        best_hidden_state_path = [hid_st_dict[i] for i in best_path]
        
        return best_hidden_state_path

In [306]:
def test_user_case_one():
    """_summary_
    """
    # index annotation observation_states=[i,j]    
    observation_states = ['on-time','late'] 

    # index annotation hidden_states=[i,j]
    hidden_states = ['no-traffic','traffic']

    # PONDERING QUESTION: How would a user define/compute their own HMM instantiation inputs to decode the hidden states for their use case observations?
    use_case_one_data = np.load('/Users/brysonchoy/Documents/hw6-hmm-viterbi/data/UserCase-One.npz')

    # Instantiate submodule class models.HiddenMarkovModel with
    # observation and hidden states and prior, transition, and emission probabilities.
    use_case_one_hmm = HiddenMarkovModel(observation_states,
                                         hidden_states,
                      use_case_one_data['prior_probabilities'], # prior probabilities of hidden states in the order specified in the hidden_states list
                      use_case_one_data['transition_probabilities'], # transition_probabilities[:,hidden_states[i]]
                      use_case_one_data['emission_probabilities']) # emission_probabilities[hidden_states[i],:][:,observation_states[j]]
    
    # Instantiate submodule class models.ViterbiAlgorithm using the use case one HMM 
    use_case_one_viterbi = ViterbiAlgorithm(use_case_one_hmm)

     # Check if use case one HiddenMarkovAlgorithm instance is inherited in the subsequent ViterbiAlgorithm instance
    assert use_case_one_viterbi.hmm_object.observation_states == use_case_one_hmm.observation_states
    assert use_case_one_viterbi.hmm_object.hidden_states == use_case_one_hmm.hidden_states

    assert np.allclose(use_case_one_viterbi.hmm_object.prior_probabilities, use_case_one_hmm.prior_probabilities)
    assert np.allclose(use_case_one_viterbi.hmm_object.transition_probabilities, use_case_one_hmm.transition_probabilities)
    assert np.allclose(use_case_one_viterbi.hmm_object.emission_probabilities, use_case_one_hmm.emission_probabilities)

    # TODO: Check HMM dimensions and ViterbiAlgorithm
    
    # Find the best hidden state path for our observation states
    use_case_decoded_hidden_states = use_case_one_viterbi.best_hidden_state_sequence(use_case_one_data['observation_states'])
    print("viterbi case: ", use_case_decoded_hidden_states)
    
    print("hidden: ", use_case_one_data['hidden_states'])
    assert np.alltrue(use_case_decoded_hidden_states == use_case_one_data['hidden_states'])

In [307]:
test_user_case_one()

obs:  ['on-time' 'on-time' 'late' 'late' 'late' 'on-time']
best path:  [0. 0. 0. 0. 0. 0.]
best path:  [0. 0. 0. 0. 0. 0.]
best path:  [0. 0. 1. 0. 0. 0.]
best path:  [0. 0. 1. 1. 0. 0.]
best path:  [0. 0. 1. 1. 0. 0.]
final path:  [[0. 1.]
 [0. 0.]
 [0. 0.]
 [0. 1.]
 [0. 1.]
 [1. 1.]]
viterbi case:  ['no-traffic', 'no-traffic', 'traffic', 'traffic', 'traffic', 'no-traffic']
hidden:  ['no-traffic' 'no-traffic' 'traffic' 'traffic' 'traffic' 'no-traffic']


In [192]:
x[0,:] = [0,0,0]

In [34]:
x

array([[0., 0., 0.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [38]:
tr = np.append(x,[4,4,4])

In [39]:
tr

array([0., 0., 0., 1., 1., 1., 1., 1., 1., 4., 4., 4.])

In [41]:
np.append(tr, 8)

array([0., 0., 0., 1., 1., 1., 1., 1., 1., 4., 4., 4., 8.])

In [200]:
test = np.random.choice(10,(10,2))
test2 = np.random.choice(10,(1,2))
print(test)
print(test2)

[[1 4]
 [7 4]
 [8 8]
 [8 0]
 [9 8]
 [2 3]
 [3 8]
 [1 5]
 [4 3]
 [5 1]]
[[4 3]]


In [154]:
np.multiply(test2,test)

array([[ 6,  0],
       [ 6, 36]])

In [203]:
np.max(test,axis=1)

array([4, 7, 8, 8, 9, 3, 8, 5, 4, 5])

In [160]:
0.132*0.4

0.05280000000000001

In [308]:
np.sum(test,axis=1)

array([ 5, 11, 16,  8, 17,  5, 11,  6,  7,  6])

In [313]:
np.allclose(np.ones((test.shape[0])),np.ones((test.shape[0])))

True